### 126. Word Ladder II

In [1]:
beginWord = "hit"; endWord = "cog"; wordList = ["hot","dot","dog","lot","log","cog"]
# Output: [["hit","hot","dot","dog","cog"],["hit","hot","lot","log","cog"]]

beginWord = "hot"; endWord = "dog"; wordList = ["hot","cog","dog","tot","hog","hop","pot","dot"]
#[['hot', 'hog', 'dog'], ['hot', 'dot', 'dog']]

In [2]:
from collections import defaultdict, deque

class Solution:
    def findLadders(self, beginWord: str, endWord: str, wordList: list):   
        print(f"beginWord = {beginWord}, endWord = {endWord}, wordList = {wordList}")   

        # 如果結束單字不在給定的wordList中，則無法找到轉換路徑，返回空列表
        if endWord not in wordList:
            return []
              
        # 建立通用狀態，將每個 wordList 中的單詞的每個位置替換成 "*" ，並將具有相同通用狀態的單字進行分組
        self.all_combo = defaultdict(list)
        for word in wordList:
            for i in range(len(word)):
                self.all_combo[word[:i] + "*" + word[i+1:]].append(word)
        print(f"all_combo = {self.all_combo}")

        # 儲存從起始單字出發的所有可能路徑
        self.visited_start = defaultdict(list)
        queue_start = deque([beginWord]) # 使用 deque 實現的起始單字的BFS
        self.visited_start[beginWord] = [] # 將起始單字設置為已訪問，並儲存起始單字的前一個節點（空列表表示起始節點）

        # 儲存從結束單字出發的所有可能路徑
        self.visited_end = defaultdict(list)
        queue_end = deque([endWord]) # 使用 deque 實現的結束單字的BFS
        self.visited_end[endWord] = [] # 將結束單字設置為已訪問，並儲存結束單字的前一個節點（空列表表示結束節點）

        self.ans = [] # 儲存最終答案


        # queue_start和queue_end都有值，且兩個BFS還未交叉找到答案時，持續搜索
        while (queue_start) and (queue_end) and (not self.ans):
            print("_" * 150)
            print(f"-> queue_start = {queue_start}")
            print(f"-> queue_end   = {queue_end}")
            print(f"-> visited_start = {self.visited_start}")
            print(f"-> visited_end   = {self.visited_end}") 
            print(f"-> ans = {self.ans}")
            
            # 選擇較小的搜索空間中進行BFS
            if len(queue_start) <= len(queue_end):
                print(f"(len(queue_start) = {len(queue_start)}) <= (len(queue_end) = {len(queue_end)})")
                print("<< BFS From Starting >>")
                self.bfs(queue = queue_start, visited_start1 = self.visited_start, visited_end1 = self.visited_end,   frombegin = True)  # 從起始單字出發的BFS
            else:
                print(f"(len(queue_start) = {len(queue_start)}) > (len(queue_end) = {len(queue_end)})")
                print("<< BFS From Ending >>")
                self.bfs(queue = queue_end,   visited_start1 = self.visited_end,   visited_end1 = self.visited_start, frombegin = False) # 從結束單字出發的BFS

        return self.ans


    # 搜尋可能的轉換路徑
    def bfs(self, queue, visited_start1, visited_end1, frombegin):
        parent_visited = defaultdict(list)  # 儲存當前層級的訪問節點及其父節點
        for _ in range(len(queue)):
            print("=" * 150)
            print(f"queue = {queue}")
            current = queue.popleft()
            print(f"current = {current}")  

            # 遍歷當前節點的所有通用狀態
            for i in range(len(current)):
                temp = current[:i] + "*" + current[i+1:]
                print(f"temp = {temp}, all_combo[{temp}] = {self.all_combo[temp]}")
                for value in self.all_combo[temp]:
                    print(f"- current = {current}, value = {value}")

                    # 如果在 visited_end 中找到了該節點，表示visited_start和visited_end兩個方向都找到同一個單字，則建立所有可能的路徑
                    if value in visited_end1:
                        print(f"- (value = {value}) in (visited_end1 = {visited_end1})") 
                        paths_start = []
                        paths_end   = []
                        print("<< DFS From Starting >>")
                        self.dfs(value = current, visited = visited_start1, path = [], paths = paths_start)
                        print("<< DFS From Ending >>")
                        self.dfs(value = value,   visited = visited_end1,   path = [], paths = paths_end)

                        if not frombegin:
                            print(f"- not (frombegin = {frombegin}) -> exchange paths")
                            paths_start, paths_end = paths_end, paths_start  # 如果是從結束單字出發，則交換 paths_start 和 paths_end

                        print(f"- paths_start = {paths_start}, paths_end = {paths_end}")

                        # 組合路徑
                        for a in paths_start:
                            for b in paths_end:
                                self.ans.append(a + b)  # 將所有可能的路徑組合添加到答案中
                                print(f"ans = {self.ans}")
                    
                    # 如果在 visited_end 中沒找到該節點，則新增節點進queue、parent_visited、visited_start
                    elif value not in visited_start1:
                        print(f"- (value = {value}) not in (visited_start1 = {visited_start1})") 
                        if value not in parent_visited:                                
                            print(f"- (value = {value}) not in (parent_visited = {parent_visited})") 
                            queue.append(value) # 如果該節點未被訪問過，則將其添加到下一層的搜索中
                            print(f"- queue = {queue}")

                        parent_visited[value].append(current) # 將當前節點添加到下一層的父節點列表中
                        print(f"- parent_visited = {parent_visited}")

        visited_start1.update(parent_visited)  # 更新 visited_start 的訪問狀態
        print(f"visited_start1 = {visited_start1}")


    # 建立所有可能的轉換路徑
    def dfs(self, value, visited, path, paths):
        print("-" * 150)
        path.append(value)
        print(f"path = {path}")

        if not visited[value]: # 如果已經到 beginWord 或 endWord，判斷是從起始單字出發還是結束單字出發
            print(f"- not (visited[value] = visited[{value}] = {visited[value]})")
            if visited == self.visited_start:
                print(f"- (visited = {visited}) == (visited_start = {self.visited_start})")
                paths.append(path[::-1]) # 如果是 visited_start 的路徑，表示從起始單字出發的DFS，反轉後添加到 paths 中
            else:
                # (path[:]是一種shallow copy，避免後面pop去改到存進paths的內容)     
                paths.append(path[:]) # 如果是 visited_end 的路徑，表示從結束單字出發的DFS，直接添加到 paths 中
            print(f"- paths = {paths}")

        print(f"visited[value] = visited[{value}] = {visited[value]}")
        for u in visited[value]: # 可能有多個父節點
            print(f"u = {u}")
            self.dfs(u, visited, path, paths)

        path.pop()
        print(f"({value} pop) - path = {path}")

In [3]:
Solution().findLadders(beginWord, endWord, wordList)

beginWord = hit, endWord = cog, wordList = ['hot', 'dot', 'dog', 'lot', 'log', 'cog']
all_combo = defaultdict(<class 'list'>, {'*ot': ['hot', 'dot', 'lot'], 'h*t': ['hot'], 'ho*': ['hot'], 'd*t': ['dot'], 'do*': ['dot', 'dog'], '*og': ['dog', 'log', 'cog'], 'd*g': ['dog'], 'l*t': ['lot'], 'lo*': ['lot', 'log'], 'l*g': ['log'], 'c*g': ['cog'], 'co*': ['cog']})
______________________________________________________________________________________________________________________________________________________
-> queue_start = deque(['hit'])
-> queue_end   = deque(['cog'])
-> visited_start = defaultdict(<class 'list'>, {'hit': []})
-> visited_end   = defaultdict(<class 'list'>, {'cog': []})
-> ans = []
(len(queue_start) = 1) <= (len(queue_end) = 1)
<< BFS From Starting >>
queue = deque(['hit'])
current = hit
temp = *it, all_combo[*it] = []
temp = h*t, all_combo[h*t] = ['hot']
- current = hit, value = hot
- (value = hot) not in (visited_start1 = defaultdict(<class 'list'>, {'hit': []}))
- 

[['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]